In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document
#from langchain_community.vectorstores import 
from langchain.output_parsers import StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import Runnable
from langchain_community.vectorstores import FAISS
from PyPDF2 import PdfReader
from dotenv import load_dotenv
from langchain.load import dumps, loads
import os

In [7]:
os.environ['OPENAI_API_KEY'] ="sk-proj-8ERQX8RyZD9FLU3ezgd0T3BlbkFJWcjvcJMdwLwli34ay6GD"

In [28]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
K = 3  

"""Loads environment variables from a .env file."""
#load_dotenv()

def create_embedding_model():
    """Creates an instance of the OpenAI embedding model."""
    return OpenAIEmbeddings()

# Extract text from pdf document


In [29]:
def Extract_pdf(pdf_reader):
    """
    Args:
        pdf_reader (PyPDF2.PdfReader): The PDF reader object containing the document content.
    Returns:
        Text extracted from pdf
    """
    pdf_reader=PdfReader(pdf_reader)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()  # Extract text from each page and concatenate
    return text

In [52]:
def create_vector_database(text):
    """
    Creates a FAISS vector database from the provided text object.   

    Returns:
        FAISS: The created FAISS vector database.
    """
    text_splitter = CharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
       # length_function=len
    )

    chunks = [Document(page_content=x) for x in text_splitter.split_text(text)]
    vectorstore = FAISS.from_documents(documents=chunks, 
                                        embedding=create_embedding_model())
    #retriever = vectorstore.as_retriever() 
    return vectorstore

In [54]:

"""Optimize and tune LLM with Multi Query method of RAG (Query Transformation) """

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Function is used to generate multiple questions from a single question
def generate_queries():
    template = """You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Original question: {question}"""
    promptprespectives = ChatPromptTemplate.from_template(template)
    generate_queries= (promptprespectives|llm|StrOutputParser()|(lambda x:x.split("\n")))
    return generate_queries

# Funcion to get unique documents
def get_unique_union(documents:list[list]):
    """Unique union of retrieved docs"""
    # Flatten list of lists and convers each Document into string
    flattened_docs =[dumps(doc) for sublist in documents for doc in sublist]
    # Get Unique documents
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]


In [55]:
question="Explain EM algorithm"

In [49]:
generate_questions=generate_queries(question)

TypeError: generate_queries() missing 1 required positional argument: 'question'

In [45]:
file_path="C:\\Users\\ksais\\Documents\\Coding\\LLMS\\Ayurveda ChatBot\\Demo.pdf"
text1=Extract_pdf(file_path)

In [46]:
vectordatabase=create_vector_database(text1)

In [47]:
vectordatabase

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B166F03290>)

In [56]:

# print(text1)
# demo_retirever = create_vector_database(text1)
# print(demo_retirever)
# gen_queries= generate_queries(template,"Who is the ceo of Microsoft?")
# print(gen_queries)

retrieval_chain = generate_queries | create_vector_database | get_unique_union

TypeError: unsupported operand type(s) for |: 'function' and 'function'

In [51]:
docs = retrieval_chain.invoke({"question":question})

TypeError: expected string or buffer